### Version simple (compter le nombre de mots de l'offre dans le CV)

In [32]:
import re
import os
from docx import Document
from collections import Counter
from nltk.corpus import stopwords
import nltk
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Téléchargement des stopwords français si nécessaire
nltk.download('stopwords')
stop_fr = set(stopwords.words('french'))

def lire_docx(path):
    """Extrait le texte brut d'un fichier Word (.docx)."""
    doc = Document(path)
    full_text = [p.text for p in doc.paragraphs]
    return "\n".join(full_text)

# --- Nettoyage de texte ---
def nettoyer_texte(texte: str) -> str:
    """Met en minuscules et supprime caractères spéciaux."""
    texte = texte.lower()
    texte = re.sub(r"[^a-zàâçéèêëîïôûùüÿñæœ0-9\s]", " ", texte)
    texte = re.sub(r"\s+", " ", texte)
    return texte.strip()

# --- Extraction des mots clés d'une offre (stop words exclus) ---
def extraire_mots_cles_offre(offre_text: str):
    """
    Retourne la liste des mots clés uniques de l'offre après nettoyage
    et suppression des stop words français.
    """
    texte = nettoyer_texte(offre_text)
    mots = texte.split()
    # Exclusion des stop words
    mots_cles = [mot for mot in mots if mot not in stop_fr]
    return list(set(mots_cles))  # mots uniques

# --- Score de correspondance CV vs offre ---
def score_cv_offre(cv_text: str, offre_text: str):
    """
    Calcule un score de correspondance CV vs offre basé à la fois sur la présence 
    et la fréquence des mots-clés de l'offre dans le CV.

    Contrairement à la version simple, cette fonction prend en compte plusieurs 
    occurrences d'un mot-clé dans le CV tout en limitant l'impact d'un mot très 
    répété grâce à `max_occurrences`.

    Algorithme :
    1. Nettoyage du texte du CV (minuscules, suppression de ponctuation, etc.).
    2. Séparation du texte en mots et comptage des occurrences de chaque mot via Counter.
    3. Extraction des mots-clés uniques de l'offre (après nettoyage).
    4. Pour chaque mot-clé de l'offre, ajouter au score le nombre d’occurrences 
       présentes dans le CV, limité par `max_occurrences` pour éviter qu’un mot
       unique répété 100 fois domine le score.
    5. Normalisation : le score total est divisé par le score maximum possible 
       (nombre de mots-clés * max_occurrences) pour obtenir un pourcentage.
    
    Arguments :
    - cv_text (str) : texte complet du CV.
    - offre_text (str) : texte complet de l'offre.
    - max_occurrences (int, optionnel) : nombre maximal d’occurrences par mot-clé
      comptabilisées pour le score (défaut 2).

    Retour :
    - score_pct (float) : pourcentage de correspondance entre le CV et l'offre, 
      basé sur la diversité et la fréquence des mots-clés.

    Exemple :
    >>> score_cv_frequence(cv_text, offre_text)
    62.5
    """
    cv_clean = nettoyer_texte(cv_text)
    cv_mots = set(cv_clean.split())

    mots_cles = extraire_mots_cles_offre(offre_text)
    # print("mots_cles:", mots_cles)

    nb_trouves = sum(1 for mot in mots_cles if mot in cv_mots)
    Score = nb_trouves / len(mots_cles) if mots_cles else 0

    return Score * 100  # Score en pourcentage

def score_cv_frequence(cv_text: str, offre_text: str, max_occurrences=2):
    """
    Score basé sur la présence et la fréquence des mots-clés contrairement à
    la fonction précedente.
    max_occurrences limite l'impact d'un mot répété.
    """
    cv_clean = nettoyer_texte(cv_text)
    cv_mots = cv_clean.split()
    cv_counts = Counter(cv_mots)

    mots_cles = extraire_mots_cles_offre(offre_text)

    score_total = 0
    for mot in mots_cles:
        score_total += min(cv_counts.get(mot, 0), max_occurrences)

    max_score = len(mots_cles) * max_occurrences
    score_pct = (score_total / max_score) * 100 if max_score > 0 else 0

    return score_pct

offre = """
Dans le cadre de sa mission d’exploitation et de valorisation des données médicales, la DIDM fait face à un besoin croissant de données fiables. C’est pourquoi un nouveau poste est créé.
Vous viendrez compléter une équipe composée d’une Chargée d’études et développements à 50 % et d’un Responsable Etudes et Développements. Sous la responsabilité de ce dernier, vos missions seront les suivantes :
Construire des pipelines de données pour alimenter la BI et l’analytique.
Modéliser et structurer les flux, tables et schémas
Garantir la qualité, la fiabilité et la sécurité des données
Développer de nouveaux datasets pour la BI de la DIDM
Mettre en place des standards de développement et de bonnes pratiques
Assurer le support et la résolution des incidents sur votre périmètre...

Votre boîte à outils
Excellente maîtrise de SQL (Oracle) et solide expérience en R
Connaissances en Julia, Java ou Scala appréciées
Pratique des outils de versioning (Git, Bitbucket, Github)
Expérience avec un outil ETL, idéalement Talend
Une première approche de la dataviz (Tableau, QlikView) est un atout
"""

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eupho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Critères = présence ou absence de mot-clé (osef de la fréquence)

In [ ]:
# Test sur un CV individuel
# Lis un CV .docx
cv_text = lire_docx("test.docx")

Score = score_cv_offre(cv_text, offre)
print(f"Score de correspondance : {Score:.1f} %")

Score de correspondance : 46.6 %


In [ ]:
# Test sur plusieurs CV 
# Faire une boucle sur les CV dans le dossier CVs
dossier_cvs = "CVs"
all_scores = {}
for nom_fichier in os.listdir(dossier_cvs):
    # Lis un CV .docx
    cv_text = lire_docx(f"./{dossier_cvs}/{nom_fichier}")

    Score = score_cv_offre(cv_text, offre)
    all_scores[nom_fichier] = Score
    print(f"Score de correspondance pour {nom_fichier} : {Score:.1f} %")

Score de correspondance pour CV - Laurent D._14_01_2024.docx : 42.0 %
Score de correspondance pour CV_AnaA_20250226.docx : 31.8 %
Score de correspondance pour CV_CSA_NRJBI_20251016.docx : 28.4 %
Score de correspondance pour NRJBI_CEC_CV_Senior.docx : 17.0 %
Score de correspondance pour NRJBI_CEC_CV_Senior_data_engineer_gemini-2.0-flash_enhanced.docx : 9.1 %
Score de correspondance pour NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash-lite_enhanced.docx : 20.5 %
Score de correspondance pour NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash_enhanced.docx : 29.5 %
Score de correspondance pour NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-pro_enhanced.docx : 39.8 %
Score de correspondance pour NRJBI_CV_EMO_202510_revisionElise.docx : 23.9 %
Score de correspondance pour NRJBI_ERE_CV - 20250930.docx : 35.2 %
Score de correspondance pour NRJBI_ERE_CV - 20250930_data_engineer_gemini-2.5-flash-lite_enhanced.docx : 34.1 %
Score de correspondance pour NRJBI_ERE_CV - 20250930_data_engineer_gemin

In [ ]:
# Convertir all_scores en DataFrame pandas pour analyse ultérieure
df_scores = pd.DataFrame(list(all_scores.items()), columns=['Fichier', 'Score'])

# Tri par ordre croissant du Score
df_scores = df_scores.sort_values(by='Score', ascending=False)
df_scores

,Fichier,Score
14,test.docx,46.590909
0,CV - Laurent D._14_01_2024.docx,42.045455
7,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-pro_enhanced.docx,39.772727
13,NRJBI_ERE_CV - 20250930_data_engineer_gemini-2.5-pro_enhanced.docx,38.636364
11,NRJBI_ERE_CV - 20250930_data_engineer_gemini-2.5-flash_enhanced.docx,37.500000
12,NRJBI_ERE_CV - 20250930_data_engineer_gemini-2.5-flash_enhanced_surligne.docx,37.500000
9,NRJBI_ERE_CV - 20250930.docx,35.227273
10,NRJBI_ERE_CV - 20250930_data_engineer_gemini-2.5-flash-lite_enhanced.docx,34.090909
1,CV_AnaA_20250226.docx,31.818182
6,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash_enhanced.docx,29.545455


In [ ]:
# Tri par ordre alphabétique des fichiers
df_scores = df_scores.sort_values(by='Fichier', ascending=True)
df_scores

,Fichier,Score
0,CV - Laurent D._14_01_2024.docx,42.045455
1,CV_AnaA_20250226.docx,31.818182
2,CV_CSA_NRJBI_20251016.docx,28.409091
3,NRJBI_CEC_CV_Senior.docx,17.045455
4,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.0-flash_enhanced.docx,9.090909
5,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash-lite_enhanced.docx,20.454545
6,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash_enhanced.docx,29.545455
7,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-pro_enhanced.docx,39.772727
8,NRJBI_CV_EMO_202510_revisionElise.docx,23.863636
9,NRJBI_ERE_CV - 20250930.docx,35.227273


In [27]:
# Affichage par ordre croissant pour chaque groupe de CVs communs (CV original + CV améliorés)
# Liste connue des fichiers originaux
original_files = [
    "NRJBI_CEC_CV_Senior.docx",
    "NRJBI_ERE_CV - 20250930.docx",
    "NRJBI_CV_EMO_202510_revisionElise.docx",
]

# Extraire juste le nom sans extension pour faciliter la recherche
original_bases = [f.rsplit('.', 1)[0] for f in original_files]

# Trouver à quel original correspond chaque fichier
def find_base(Fichier):
    for base in original_bases:
        if Fichier.startswith(base):  # on compare le début du nom
            return base
    return Fichier  # si aucun match, on garde le nom lui-même

df_scores["base_name"] = df_scores["Fichier"].apply(find_base)

# S'il manque l'extension .docx dans base_name, on l'ajoute pour correspondre aux clés originales
df_scores["base_name"] = df_scores["base_name"].apply(lambda x: x + ".docx" if not x.endswith(".docx") else x)

# Trier : d’abord par base_name, puis par Score décroissant
df_sorted = df_scores.sort_values(["base_name", "Score"], ascending=[True, False]).reset_index(drop=True)

# On crée un dictionnaire base_name -> score original
original_score_dict = df_scores.set_index("Fichier").loc[original_files, "Score"].to_dict()

# Ajouter la colonne original_score à tous les fichiers selon leur base_name
df_sorted["original_score"] = df_sorted["base_name"].map(original_score_dict)

# Calculer le gain par rapport à l’original
df_sorted["gain_vs_original"] = df_sorted["Score"] - df_sorted["original_score"]

# Créer une colonne booléenne : True si c'est le CV original
df_sorted["is_original"] = df_sorted["Fichier"].isin(original_files)

# Trier : d'abord par base_name, puis par is_original (True en premier), puis par Score décroissant
df_sorted = df_sorted.sort_values(
    by=["base_name", "is_original", "Score"],
    ascending=[True, False, False]
).reset_index(drop=True)

# Supprimer la colonne temporaire si nécessaire
df_sorted = df_sorted.drop(columns="is_original")

# Affichage lisible
for base, group in df_sorted.groupby("base_name"):
    print(f"\n📄 {base}")
    for _, r in group.iterrows():
        gain = f"(+{r['gain_vs_original']:.2f})" if r['gain_vs_original'] > 0 else f"({r['gain_vs_original']:.2f})"
        print(f"   {r['Fichier']:<80} {r['Score']:6.2f} {gain}")


📄 CV - Laurent D._14_01_2024.docx
   CV - Laurent D._14_01_2024.docx                                                   42.05 (nan)

📄 CV_AnaA_20250226.docx
   CV_AnaA_20250226.docx                                                             31.82 (nan)

📄 CV_CSA_NRJBI_20251016.docx
   CV_CSA_NRJBI_20251016.docx                                                        28.41 (nan)

📄 NRJBI_CEC_CV_Senior.docx
   NRJBI_CEC_CV_Senior.docx                                                          17.05 (0.00)
   NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-pro_enhanced.docx                    39.77 (+22.73)
   NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash_enhanced.docx                  29.55 (+12.50)
   NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash-lite_enhanced.docx             20.45 (+3.41)
   NRJBI_CEC_CV_Senior_data_engineer_gemini-2.0-flash_enhanced.docx                   9.09 (-7.95)

📄 NRJBI_CV_EMO_202510_revisionElise.docx
   NRJBI_CV_EMO_202510_revisionElise.docx            

In [24]:
df_sorted

,Fichier,Score,base_name,original_score,gain_vs_original
0,CV - Laurent D._14_01_2024.docx,42.045455,CV - Laurent D._14_01_2024.docx,NaN,NaN
1,CV_AnaA_20250226.docx,31.818182,CV_AnaA_20250226.docx,NaN,NaN
2,CV_CSA_NRJBI_20251016.docx,28.409091,CV_CSA_NRJBI_20251016.docx,NaN,NaN
3,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-pro_enhanced.docx,39.772727,NRJBI_CEC_CV_Senior,NaN,NaN
4,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash_enhanced.docx,29.545455,NRJBI_CEC_CV_Senior,NaN,NaN
5,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash-lite_enhanced.docx,20.454545,NRJBI_CEC_CV_Senior,NaN,NaN
6,NRJBI_CEC_CV_Senior.docx,17.045455,NRJBI_CEC_CV_Senior,NaN,NaN
7,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.0-flash_enhanced.docx,9.090909,NRJBI_CEC_CV_Senior,NaN,NaN
8,NRJBI_CV_EMO_202510_revisionElise.docx,23.863636,NRJBI_CV_EMO_202510_revisionElise,NaN,NaN
9,NRJBI_ERE_CV - 20250930_data_engineer_gemini-2.5-pro_enhanced.docx,38.636364,NRJBI_ERE_CV - 20250930,NaN,NaN


In [21]:
df_scores

,Fichier,Score,base_name
0,CV - Laurent D._14_01_2024.docx,42.045455,CV - Laurent D._14_01_2024.docx
1,CV_AnaA_20250226.docx,31.818182,CV_AnaA_20250226.docx
2,CV_CSA_NRJBI_20251016.docx,28.409091,CV_CSA_NRJBI_20251016.docx
3,NRJBI_CEC_CV_Senior.docx,17.045455,NRJBI_CEC_CV_Senior
4,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.0-flash_enhanced.docx,9.090909,NRJBI_CEC_CV_Senior
5,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash-lite_enhanced.docx,20.454545,NRJBI_CEC_CV_Senior
6,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash_enhanced.docx,29.545455,NRJBI_CEC_CV_Senior
7,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-pro_enhanced.docx,39.772727,NRJBI_CEC_CV_Senior
8,NRJBI_CV_EMO_202510_revisionElise.docx,23.863636,NRJBI_CV_EMO_202510_revisionElise
9,NRJBI_ERE_CV - 20250930.docx,35.227273,NRJBI_ERE_CV - 20250930


#### Critères = présence ou absence + fréquence de mot-clés

In [30]:
# Test sur plusieurs CV 
# Faire une boucle sur les CV dans le dossier CVs
dossier_cvs = "CVs"
all_scores = {}
for nom_fichier in os.listdir(dossier_cvs):
    # Lis un CV .docx
    cv_text = lire_docx(f"./{dossier_cvs}/{nom_fichier}")
    Score = score_cv_frequence(cv_text, offre, max_occurrences=3)
    all_scores[nom_fichier] = Score
    print(f"Score de correspondance pour {nom_fichier} : {Score:.1f} %")

Score de correspondance pour CV - Laurent D._14_01_2024.docx : 29.5 %
Score de correspondance pour CV_AnaA_20250226.docx : 22.3 %
Score de correspondance pour CV_CSA_NRJBI_20251016.docx : 16.7 %
Score de correspondance pour NRJBI_CEC_CV_Senior.docx : 12.1 %
Score de correspondance pour NRJBI_CEC_CV_Senior_data_engineer_gemini-2.0-flash_enhanced.docx : 3.8 %
Score de correspondance pour NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash-lite_enhanced.docx : 14.8 %
Score de correspondance pour NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash_enhanced.docx : 20.1 %
Score de correspondance pour NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-pro_enhanced.docx : 25.0 %
Score de correspondance pour NRJBI_CV_EMO_202510_revisionElise.docx : 17.0 %
Score de correspondance pour NRJBI_ERE_CV - 20250930.docx : 22.3 %
Score de correspondance pour NRJBI_ERE_CV - 20250930_data_engineer_gemini-2.5-flash-lite_enhanced.docx : 19.7 %
Score de correspondance pour NRJBI_ERE_CV - 20250930_data_engineer_gemin

In [31]:
# Affichage par ordre croissant pour chaque groupe de CVs communs (CV original + CV améliorés)
# Liste connue des fichiers originaux
original_files = [
    "NRJBI_CEC_CV_Senior.docx",
    "NRJBI_ERE_CV - 20250930.docx",
    "NRJBI_CV_EMO_202510_revisionElise.docx",
]

# Extraire juste le nom sans extension pour faciliter la recherche
original_bases = [f.rsplit('.', 1)[0] for f in original_files]

# Trouver à quel original correspond chaque fichier
def find_base(Fichier):
    for base in original_bases:
        if Fichier.startswith(base):  # on compare le début du nom
            return base
    return Fichier  # si aucun match, on garde le nom lui-même

df_scores["base_name"] = df_scores["Fichier"].apply(find_base)

# S'il manque l'extension .docx dans base_name, on l'ajoute pour correspondre aux clés originales
df_scores["base_name"] = df_scores["base_name"].apply(lambda x: x + ".docx" if not x.endswith(".docx") else x)

# Trier : d’abord par base_name, puis par Score décroissant
df_sorted = df_scores.sort_values(["base_name", "Score"], ascending=[True, False]).reset_index(drop=True)

# On crée un dictionnaire base_name -> score original
original_score_dict = df_scores.set_index("Fichier").loc[original_files, "Score"].to_dict()

# Ajouter la colonne original_score à tous les fichiers selon leur base_name
df_sorted["original_score"] = df_sorted["base_name"].map(original_score_dict)

# Calculer le gain par rapport à l’original
df_sorted["gain_vs_original"] = df_sorted["Score"] - df_sorted["original_score"]

# Créer une colonne booléenne : True si c'est le CV original
df_sorted["is_original"] = df_sorted["Fichier"].isin(original_files)

# Trier : d'abord par base_name, puis par is_original (True en premier), puis par Score décroissant
df_sorted = df_sorted.sort_values(
    by=["base_name", "is_original", "Score"],
    ascending=[True, False, False]
).reset_index(drop=True)

# Supprimer la colonne temporaire si nécessaire
df_sorted = df_sorted.drop(columns="is_original")

# Affichage lisible
for base, group in df_sorted.groupby("base_name"):
    print(f"\n📄 {base}")
    for _, r in group.iterrows():
        gain = f"(+{r['gain_vs_original']:.2f})" if r['gain_vs_original'] > 0 else f"({r['gain_vs_original']:.2f})"
        print(f"   {r['Fichier']:<80} {r['Score']:6.2f} {gain}")


📄 CV - Laurent D._14_01_2024.docx
   CV - Laurent D._14_01_2024.docx                                                   42.05 (nan)

📄 CV_AnaA_20250226.docx
   CV_AnaA_20250226.docx                                                             31.82 (nan)

📄 CV_CSA_NRJBI_20251016.docx
   CV_CSA_NRJBI_20251016.docx                                                        28.41 (nan)

📄 NRJBI_CEC_CV_Senior.docx
   NRJBI_CEC_CV_Senior.docx                                                          17.05 (0.00)
   NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-pro_enhanced.docx                    39.77 (+22.73)
   NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash_enhanced.docx                  29.55 (+12.50)
   NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash-lite_enhanced.docx             20.45 (+3.41)
   NRJBI_CEC_CV_Senior_data_engineer_gemini-2.0-flash_enhanced.docx                   9.09 (-7.95)

📄 NRJBI_CV_EMO_202510_revisionElise.docx
   NRJBI_CV_EMO_202510_revisionElise.docx            

### Version tout le CV offre transformés en vecteurs

In [23]:
# --- REQUIREMENTS ---
# pip install scikit-learn nltk
# ---------------------------------------------

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk, os, re
from docx import Document

# Téléchargement du stopword français
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_fr = stopwords.words('french')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eupho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# --- Fonctions ---

def lire_docx(path):
    """Extrait le texte brut d'un fichier Word (.docx)."""
    doc = Document(path)
    full_text = [p.text for p in doc.paragraphs]
    return "\n".join(full_text)

def nettoyer_texte(texte: str) -> str:
    """Nettoie et normalise un texte français."""
    texte = texte.lower()
    texte = re.sub(r"[^a-zàâçéèêëîïôûùüÿñæœ0-9\s]", " ", texte)
    texte = re.sub(r"\s+", " ", texte)
    texte = re.sub(r"\d+", " ", texte)  # supprime tous les nombres
    return texte.strip()

def score_cv_contre_offre(cv_text, offre_text):
    """Calcule le Score TF-IDF entre un CV unique et l'offre."""
    # Nettoyage des textes
    cv_clean = nettoyer_texte(cv_text)
    offre_clean = nettoyer_texte(offre_text)

    # TF-IDF
    vectorizer = CountVectorizer(stop_words=stop_fr, ngram_range=(1,1))
    count_words = vectorizer.fit_transform([offre_clean, cv_clean])

    # Vecteurs
    offre_vec = count_words[0:1]
    cv_vec = count_words[1:2]

    # Similarité cosinus
    Score = cosine_similarity(offre_vec, cv_vec)[0][0]
    return Score

In [ ]:
# Lis l'offre
offre_text = """Dans le cadre de sa mission d’exploitation et de valorisation des données médicales, 
la DIDM fait face à un besoin croissant de données fiables. Vous viendrez compléter une équipe...
Excellente maîtrise de SQL (Oracle) et solide expérience en R...
Pratique des outils de versioning (Git, Bitbucket, Github)
Expérience avec un outil ETL, idéalement Talend
Une première approche de la dataviz (Tableau, QlikView) est un atout."""

# Lis un CV .docx
cv_text = lire_docx("test.docx")

# Calcul du Score
Score = score_cv_contre_offre(cv_text, offre_text)
print(f"Score de correspondance : {Score*100:.1f} %")

Score de correspondance : 38.9 %


In [47]:
cv_text

'Ingénieur Data / Développeur BI – Spécialisation données médicales\nEmail : jean.dupont@example.com\nTéléphone : 06 45 89 22 77\n\nProfil\nData engineer expérimenté dans l’exploitation, la fiabilité et la valorisation de données médicales.\nHabitué à modéliser des flux de données, construire des pipelines ETL complexes sous Talend et SQL (Oracle), et à assurer la qualité, la sécurité et la traçabilité des données pour des environnements analytiques et décisionnels (BI, tableaux de bord, reporting).\n\nCompétences clés\nLangages : SQL (Oracle, PostgreSQL), R, Java, Scala, Julia (bases)\nOutils ETL / Data Pipeline : Talend, Airflow, DataStage\nBusiness Intelligence : Tableau, QlikView, Power BI\nVersioning : Git, Bitbucket, GitHub\nData Modeling : conception de schémas, modélisation de tables, intégration de données hétérogènes\nQualité & Sécurité : validation des datasets, contrôles de cohérence, gestion des accès et anonymisation\nBonnes pratiques : documentation, revue de code, stand

In [18]:
scores

array([0.07637796, 0.27880457])

In [ ]:
#Scoring CV hors fonction
"""Calcule le Score TF-IDF entre un CV unique et l'offre."""
# Nettoyage des textes
cv_clean = nettoyer_texte(cv_text)
offre_clean = nettoyer_texte(offre_text)

# TF-IDF
vectorizer = CountVectorizer(stop_words=stop_fr, ngram_range=(1,1))
count_words = vectorizer.fit_transform([offre_clean, cv_clean])

# Vecteurs
offre_vec = count_words[0:1]
cv_vec = count_words[1:2]

# Similarité cosinus
Score = cosine_similarity(offre_vec, cv_vec)[0][0]
print("Score:", Score)

score: 0.3672651915581337


In [37]:
print(vectorizer.get_feature_names_out())

['accès' 'advanced' 'airflow' 'alimenter' 'analyste' 'analytique'
 'analytiques' 'anglais' 'anonymisation' 'approche' 'assurer' 'atout'
 'aujourd' 'autres' 'avancé' 'bases' 'bash' 'besoin' 'bi' 'bitbucket'
 'bonnes' 'bord' 'business' 'cadre' 'certification' 'clés' 'cnil' 'code'
 'cohérence' 'com' 'complexes' 'compléter' 'compétences' 'conception'
 'construction' 'construire' 'contraintes' 'contrôle' 'contrôles'
 'croissant' 'data' 'datasets' 'datastage' 'dataviz' 'didm'
 'documentation' 'données' 'dupont' 'décisionnels' 'département'
 'développement' 'développeur' 'email' 'engineer' 'engineering' 'entrepôt'
 'environnements' 'etl' 'example' 'excel' 'excellente' 'expert'
 'exploitation' 'expérience' 'expérimenté' 'face' 'fait' 'fiabilité'
 'fiables' 'flux' 'formation' 'foundations' 'france' 'français' 'garantir'
 'gestion' 'git' 'github' 'habitué' 'hui' 'hétérogènes' 'hôpital'
 'idéalement' 'incidents' 'information' 'informatique' 'ingénieur'
 'integration' 'intelligence' 'interactifs' 

In [28]:
print(offre_clean)

dans le cadre de sa mission d exploitation et de valorisation des données médicales la didm fait face à un besoin croissant de données fiables vous viendrez compléter une équipe excellente maîtrise de sql oracle et solide expérience en r pratique des outils de versioning git bitbucket github expérience avec un outil etl idéalement talend une première approche de la dataviz tableau qlikview est un atout


In [16]:
offre_vec

<1x1164 sparse matrix of type '<class 'numpy.float64'>'
	with 181 stored elements in Compressed Sparse Row format>

In [ ]:
count_words

<3x1164 sparse matrix of type '<class 'numpy.float64'>'
	with 1285 stored elements in Compressed Sparse Row format>

In [14]:
cleaned

['dans le cadre de sa mission d exploitation et de valorisation des données médicales la didm fait face à un besoin croissant de données fiables c est pourquoi un nouveau poste est créé vous viendrez compléter une équipe composée d une chargée d études et développements à 50 et d un responsable etudes et développements sous la responsabilité de ce dernier vos missions seront les suivantes construire des pipelines de données pour alimenter la bi et l analytique modéliser et structurer les flux tables et schémas garantir la qualité la fiabilité et la sécurité des données développer de nouveaux datasets pour la bi de la didm mettre en place des standards de développement et de bonnes pratiques assurer le support et la résolution des incidents sur votre périmètre votre boîte à outils excellente maîtrise de sql oracle et solide expérience en r connaissances en julia java ou scala appréciées pratique des outils de versioning git bitbucket github expérience avec un outil etl idéalement talend

In [12]:
print(len(cv_texts))

2


In [11]:
print(len(documents))

3


In [10]:
documents

['Dans le cadre de sa mission d’exploitation et de valorisation des données médicales, la DIDM fait face à un besoin croissant de données fiables. C’est pourquoi un nouveau poste est créé.\nVous viendrez compléter une équipe composée d’une Chargée d’études et développements à 50 % et d’un Responsable Etudes et Développements. Sous la responsabilité de ce dernier, vos missions seront les suivantes :\nConstruire des pipelines de données pour alimenter la BI et l’analytique.\nModéliser et structurer les flux, tables et schémas\nGarantir la qualité, la fiabilité et la sécurité des données\nDévelopper de nouveaux datasets pour la BI de la DIDM\nMettre en place des standards de développement et de bonnes pratiques\nAssurer le support et la résolution des incidents sur votre périmètre...\n\nVotre boîte à outils\nExcellente maîtrise de SQL (Oracle) et solide expérience en R\nConnaissances en Julia, Java ou Scala appréciées\nPratique des outils de versioning (Git, Bitbucket, Github)\nExpérience